In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import gym
import time
import random
import pybullet_envs
from pathlib import Path
from datetime import datetime 
# import DDPG
# import original_buffer
# import PER_buffer
# import utils
import matplotlib.pyplot as plt

In [ ]:
import os
import h5py
from tqdm import tqdm
from gym import spaces
from collections import deque


In [ ]:
channels = 'channels/channel_K6_AP32_2k_250x250.h5'
channel_no = 1
APs = 32
UEs = 6
# channel_K6_AP10_2k_20x20


class wirelessCF(gym.Env):
  """Custom Environment that follows gym interface"""
  metadata = {'render.modes': ['human']}

  def __init__(self, pu, nrx, gainspath=channels, channel_setup=channel_no, seed=0, eval = None):
    np.random.seed(seed)
    random.seed(seed)
    
    super(wirelessCF, self).__init__()
    

    # cf parameters
    # self.channelGains = h5py.File(base_dir+str(gainspath), 'r')
    self.channelGains = h5py.File(str(gainspath), 'r')
    self.pu = np.power(10, pu/10 - 3)
    self.B = 20000000
    self.N0 = 1
    self.T_c = 0.001
    self.Ptcm = 0.2
    self.Ptcl = 0.2
    self.Pom = 0.825
    self.pp = 0.2
    self.K = UEs
    self.tauc = 200
    self.eff = 0.4 #amplifier efficiency
    self.M = APs

    self.Pft = 10
    self.C_fh = 100000000
    self.nu = 2
    self.a = 0.88115
    self.b = 0.88115
    self.taup = self.K
    self.tauf = 1 - (self.taup/self.tauc)
    self.Nrx = nrx
    self.R_fh = 2*self.K*self.nu*self.tauf*self.tauc/self.T_c
    self.Pfix   = self.M*((self.Nrx*self.Ptcm)+self.Pom + self.Pft*self.R_fh/self.C_fh)/self.K
    self.theta_max = 1
    self.alpha = deque(maxlen=1)
    self.beta = deque(maxlen=1)
    self.ch_gain = deque(maxlen=1)
    self.wi = [1/self.K for i in range(self.K)] # define equal weights which sum to 1
    self.pi = deque(maxlen=1)
    self.reward_timediff = deque([0,0],maxlen=2)
    self.initialize_p()
    self.channel_setup = channel_setup
    self.initialize_matrix(channel_setup)
    self.max_episode_step = 100
    self._max_episode_steps = 100
    # define action and obervation space:
    self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                        shape=(self.K,), dtype=np.float32)
    MIN_ACTION = 10e-7
    self.action_space = spaces.Box(low=MIN_ACTION, high=1,
                                  shape=(self.K,), dtype=np.float32)    

    self.eval = eval

    # history parameters
    self.WSEE_history = []
    self.sumSE_history = []
    self.FPA_WSEE_history = []
    self.FPA_sumSE_history = []
    self.score_history = []
    self.step_WSEE = []
    self.step_sumSE = []
    self.step_score = []
    self.time_per_episode = []
    self.curr_time = 0
    self.end_time = 0

    self.global_step = 0

    # eval history
    # self.eval_history = {}
    # self.eval_history['WSEE'] = []
    # self.eval_history['sumSE'] = []
    # self.eval_history['FPA_WSEE'] = []
    # self.eval_history['FPA_sumSE'] = []
    # self.eval_history['score'] = []

  def initialize_matrix(self, episode):
        BETA = self.channelGains['channelGain'][episode].transpose()
        # BETA = np.random.uniform(0,10,size=(1,self.K,self.M))
        # h5f = h5py.File(str(dest)+'/rand_BETA.h5', 'w')
        # h5f.create_dataset('channelGain', data=BETA)
        # h5f.close()
        # BETA = BETA[0].transpose()
        print(BETA)
        gamma_num = np.zeros((self.M,self.K))
        gamma_den = np.zeros((self.M,self.K))

        Gamma = np.zeros((self.M,self.K))
        for m in range(self.M):
            for k in range(self.K):
                gamma_num[m][k] = self.taup*self.pp*np.power(BETA[m][k],2)                                  
                gamma_den[m][k] = self.taup*self.pp*BETA[m][k]+1                    
                Gamma[m][k] = gamma_num[m][k]/gamma_den[m][k]
        self.ch_gain.append(Gamma)
        alpha1 = np.zeros((self.K,))
        for k in range(self.K):
            #alpha1[k] = self.Nrx*self.pu*np.sum(Gamma[:][k])*self.pi[t][k]
            alpha1[k] = self.pu*np.power(self.a*self.Nrx*np.sum(Gamma[:,k]),2)
        self.alpha.append(alpha1)
        beta1 = np.zeros((self.K,self.K))
        for k in range(self.K):
            for q in range(self.K):
                beta1 [k][q] = self.a*self.a*self.pu*self.Nrx*(BETA[:,q].T@Gamma[:,k])         
        self.beta.append(beta1)
        return 
  
  def initialize_p(self):
      self.pi.append(np.random.uniform(low=0, high=self.theta_max, size=(self.K,)))
      return
    
  def cal_alpha_p(self, i, t):
      val = self.alpha[t][i]*self.pi[t][i]
      return val
  
  def cal_beta_p(self, i, j, t):
      val = self.beta[t][i][j]*self.pi[t][j] # channel from UE j to BS i
      return val
  
  def sum_beta_p(self, i, t):
      val = 0
      for j in range(self.K):
          val += self.cal_beta_p(i,j,t)  
      return val 

  def cal_SINR(self,i,t):
      val = self.cal_alpha_p(i,t)/(self.b*self.Nrx*np.sum(np.asarray(self.ch_gain)[t,:,i]) + \
                                              (self.b-self.a*self.a)*self.Nrx*self.Nrx*self.pu*self.pi[t][i]*np.linalg.norm(np.asarray(self.ch_gain)[t,:,i])**2 + \
                                              self.b/(self.a*self.a)*self.sum_beta_p(i, t))
      return val

  def cal_Ri(self,i, t):
        val = np.log2(1+(self.cal_SINR(i,t)))
        return val
      
  def cal_equal_p_Ri(self,i, t):
      p = [self.theta_max for x in range(self.K)]
      temp = self.pi.copy()
      self.pi[t] = p
      val = self.cal_Ri(i,t)
      self.pi = temp
      return val

  def cal_EEi(self,i,t):
      val = self.tauf*self.cal_Ri(i,t)/(self.pu*self.N0*self.pi[t][i]/self.eff + self.Pfix + self.Ptcl)
      return val

  def cal_total_WSEE(self,t):
      val = 0
      for x in range(self.K):
          val += self.wi[x]*self.cal_EEi(x,t)
      return val
  
  def cal_equal_p_WSEE(self, t):
      p = [self.theta_max for x in range(self.K)]
      temp = self.pi.copy()
      self.pi[t] = p
      val = self.cal_total_WSEE(t)
      self.pi = temp
      return val

  def cal_SE_vec(self,t):
      SEs = []
      for u in range(self.K):
        SE_u = self.tauf*self.cal_Ri(u,t)
        SEs.append(SE_u)
      return SEs

  def cal_sum_SE(self,t):
      SEs = self.cal_SE_vec(t)
      sum_SE = sum([se for se in SEs])
      return sum_SE

  def cal_fpa_SE_vec(self,t):
      SEs = []
      for u in range(self.K):
        SE_u = self.tauf*self.cal_equal_p_Ri(u,t)
        SEs.append(SE_u)
      return SEs

  def cal_sum_fpa_SE(self,t):
      SEs = self.cal_fpa_SE_vec(t)
      sum_SE = sum([se for se in SEs])
      return sum_SE

  def cal_reward(self,t):
      """
      SE optimization
      """
      # ri = self.cal_sum_SE(t) 
      """
      WSEE optimization
      """
      r = self.cal_total_WSEE(t) - self.cal_equal_p_WSEE(t)
    #   ri = self.cal_total_WSEE(t)
    #   self.reward_timediff.append(ri)
    #   r = self.reward_timediff[1]-self.reward_timediff[0]
      """
      SE+WSEE optimization
      """
      # r = joint_weights*(self.cal_total_WSEE(t) - self.cal_equal_p_WSEE(t)) + (1-joint_weights)*(self.cal_sum_SE(t) - self.cal_sum_fpa_SE(t))
      # r = (self.cal_total_WSEE(t) - self.cal_equal_p_WSEE(t)) + (self.cal_sum_SE(t) - self.cal_sum_fpa_SE(t))
      return r
      
  def cal_state(self):
      state = []
      for t in range(1): #for current timestep only
          for k in range(self.K):
            state.append(self.cal_SINR(k,t)) #M
          
      # state = np.reshape(state, [1, self.observation_space.shape[0]])
      normalized_states = state/np.max(state)
      # return normalized_states
      return -np.log10(state)

  def initialize_state(self):
      state = self.cal_state()
      return state

  def reset(self):
      self.curr_time = time.time()
      # self.initialize_matrix(self.channel_setup)
      self.initialize_p()
      self.episode_step = 0
      state = self.initialize_state()
      
      return state

          
  def step(self, actions):
      self.episode_step += 1
      self.global_step += 1
      # actions = actions[0] #theres an extra dim
      self.pi.append(actions)
      next_state = self.cal_state()
      reward = self.cal_reward(0)
      # next_state = np.reshape(next_state, [1, self.observation_space.shape[0]])
      done = False
      
      self.step_WSEE.append(self.cal_total_WSEE(0))
      self.step_sumSE.append(self.cal_sum_SE(0))
      self.step_score.append(reward)
      if self.episode_step >= self.max_episode_step:
          
          self.WSEE_history.append(np.mean(self.step_WSEE))
          self.sumSE_history.append(np.mean(self.step_sumSE))
          self.score_history.append(np.mean(self.step_score))
          self.FPA_WSEE_history.append(self.cal_equal_p_WSEE(0))
          self.FPA_sumSE_history.append(self.cal_sum_fpa_SE(0))


          self.step_WSEE = []
          self.step_sumSE = []
          self.step_score = []

          mode = "Training"
          if self.eval:
            mode = "Evaluation"
          print(f'  {str(mode)}: Global Step: {self.global_step} || avg_score: {self.score_history[-1]} || WSEE: {self.WSEE_history[-1]} || FPA_WSEE: {self.cal_equal_p_WSEE(0)} || sumSE: {self.sumSE_history[-1]} || FPA_sumSE: {self.cal_sum_fpa_SE(0)}')

          done = True
          self.end_time = time.time()
          self.time_per_episode.append(self.end_time - self.curr_time)
          print('Time', self.time_per_episode[-1])
          
      info = {
          'WSEE': self.cal_total_WSEE(0),
          'FPA_WSEE': self.cal_equal_p_WSEE(0),
          'sumSE': self.cal_sum_SE(0),
          'FPA_sumSE': self.cal_sum_fpa_SE(0)
      }
      return next_state, reward, done, info

  
  def render(self, mode='human'):
    NotImplementedError
  def close (self):
    NotImplementedError


In [ ]:
device = "cpu"

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import original_buffer
import PER_buffer

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
fc1_dims = 500
fc2_dims = 400
fc3_dims = 300
# fc4_dims = 30
#Architecture from experimental details section of the DDPG 
#paper "Continuous control with deep reinforcement learning"
#Lillicrap et. al. 2015
class Actor(nn.Module):
    def __init__(self, s_dim, a_dim, a_max, args):
        super(Actor, self).__init__()
        self.a_max = a_max
        
        self.l1 = nn.Linear(s_dim, args.fc1_dims)
        self.l2 = nn.Linear(args.fc1_dims, args.fc2_dims)
        self.l3 = nn.Linear(args.fc2_dims, args.fc3_dims)
        # self.l4 = nn.Linear(fc3_dims, fc4_dims)
        self.l5 = nn.Linear(args.fc3_dims, a_dim)
  
    def forward(self, s):
        x = F.relu(self.l1(s))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        # x = F.relu(self.l4(x))
        x = torch.tanh(self.l5(x)) * self.a_max  
        return x 

#Architecture from experimental details section of the DDPG
#paper "Continuous control with deep reinforcement learning"
#Lillicrap et. al. 2015
class Critic(nn.Module):
    def __init__(self, s_dim, a_dim, args):
        super(Critic, self).__init__()
        self.l1 = nn.Linear(s_dim, args.fc1_dims)
        self.l2 = nn.Linear(args.fc1_dims + a_dim, args.fc2_dims)
        self.l3 = nn.Linear(args.fc2_dims, args.fc3_dims)
        # self.l4 = nn.Linear(fc3_dims, fc4_dims)
        self.l5 = nn.Linear(args.fc3_dims, 1)

    def forward(self, s, a):
        x = F.relu(self.l1(s))
        x = torch.cat([x, a], 1)
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        # x = F.relu(self.l4(x))
        x = self.l5(x)
        return x 

class DDPG(object):
    def __init__(self, s_dim, a_dim, a_max, args):
        #Create actor and actor target 
        self.actor = Actor(s_dim, a_dim, a_max, args).to(device)
        self.actor_target = Actor(s_dim, a_dim, a_max, args).to(device)
        #Initialize actor and actor target exactly the same
        self.actor_target.load_state_dict(self.actor.state_dict())
        #Adam optimizer to train actor
        #Learning rate specified in DDPG paper
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=args.lr_a)
        
        #Create critic and critic target
        self.critic = Critic(s_dim, a_dim, args).to(device)
        self.critic_target = Critic(s_dim, a_dim, args).to(device)
        #Initialize critic and critic target exactly the same
        self.critic_target.load_state_dict(self.critic.state_dict())
        #Adam optimizer to train critic
        #L2 weight decay specified in DDPG paper
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=args.lr_c, weight_decay=1e-2)
    
    #Given a state, the actor returns a policy 
    def get_action(self, s):
        s = torch.FloatTensor(s.reshape(1, -1)).to(device)
        return self.actor(s).cpu().data.numpy().flatten()

    #Update actor, critic and target networks with minibatch of experiences
    def train(self, replay_buffer, prioritized, beta_value, epsilon, T, batch_size=64, gamma=0.99, tau=1e-4):

        for i in range(T):
            # Sample replay buffer
            if prioritized: 
                #Prioritized experience replay
                experience = replay_buffer.sample(batch_size, beta_value)
                s, a, r, s_new, done, weights, batch_idxes = experience
                #reshape data
                r = r.reshape(-1, 1)
                done = done.reshape(-1, 1)
                #We do not use importance sampling weights
                #Therefore importance sampling weights are all set to 1
                #See Hyperparameter search in report 
                weights = np.ones_like(r)
                #weights = weights.reshape(-1, 1)
            else:
                #Uniform experience replay
                s, a, r, s_new, done = replay_buffer.sample(batch_size)
                #importance sampling weights are all set to 1
                weights, batch_idxes = np.ones_like(r), None

            #Sqrt weights 
            #We do this since each weight will squared in MSE loss
            weights = np.sqrt(weights)

            #convert data to tensors
            state = torch.FloatTensor(s).to(device)
            action = torch.FloatTensor(a).to(device)
            next_state = torch.FloatTensor(s_new).to(device)
            done = torch.FloatTensor(1 - done).to(device)
            reward = torch.FloatTensor(r).to(device)
            weights = torch.FloatTensor(weights).to(device)
   
            #Compute the Q value estimate of the target network
            Q_target = self.critic_target(next_state, self.actor_target(next_state))
            #Compute Y
            Y = reward + (done * gamma * Q_target).detach()
            #Compute Q value estimate of critic
            Q = self.critic(state, action)
            #Calculate TD errors
            TD_errors = (Y - Q)
            #Weight TD errors 
            weighted_TD_errors = torch.mul(TD_errors, weights)
            #Create a zero tensor
            zero_tensor = torch.zeros(weighted_TD_errors.shape)
            #Compute critic loss, MSE of weighted TD_r
            critic_loss = F.mse_loss(weighted_TD_errors,zero_tensor)

            #Update critic by minimizing the loss
            #https://pytorch.org/docs/stable/optim.html
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Compute actor loss
            actor_loss = -self.critic(state, self.actor(state)).mean()
            #Update the actor policy using the sampled policy gradient:
            #https://pytorch.org/docs/stable/optim.html
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update the target models
            for critic_weights, critic__target_weights in zip(self.critic.parameters(), self.critic_target.parameters()):
                critic__target_weights.data.copy_(tau * critic_weights.data + (1 - tau) * critic__target_weights.data)
            for actor_weights, actor__target_weights in zip(self.actor.parameters(), self.actor_target.parameters()):
                actor__target_weights.data.copy_(tau * actor_weights.data + (1 - tau) * actor__target_weights.data)
    
            #For prioritized exprience replay
            #Update priorities of experiences with TD errors
            if prioritized:
                td_errors = TD_errors.detach().numpy()
                new_priorities = np.abs(td_errors) + epsilon
                replay_buffer.update_priorities(batch_idxes, new_priorities)


In [ ]:
import numpy as np

class ReplayBuffer_org(object):
    def __init__(self, capacity=1e6):
        self.buffer = []
        self.capacity = capacity
        #used to track which experience is to be removed next
        self.current_pos = 0

    def add(self, data):
        #If buffer is at capacity, remove the experience that has been in
        #for the longest time and add the new experience in its place
        if len(self.buffer) == self.capacity:
            self.buffer[int(self.current_pos)] = data
            self.current_pos = (self.current_pos + 1) % self.capacity
        #If buffer is not at capacity, just add experience
        else:
            self.buffer.append(data)

    def sample(self, batch_size):
        #get minibatch of indexes
        indexes = np.random.randint(0, len(self.buffer), size=batch_size)
        s, s_new, a, r, d = [], [], [], [], []
        #copy experience data from buffer into minibatch
        for index in indexes:
            states, actions, rewards, new_states, done = self.buffer[index]
            s.append(np.array(states, copy=False))
            a.append(np.array(actions, copy=False))
            r.append(np.array(rewards, copy=False))
            s_new.append(np.array(new_states, copy=False))
            d.append(np.array(done, copy=False))
        #return minibatch of experience data
        return np.array(s), np.array(a), np.array(r).reshape(-1, 1), np.array(s_new), np.array(d).reshape(-1, 1)


In [ ]:
import numpy as np
import random

import utils


class ReplayBuffer(object):
    def __init__(self, size):
        """Create Replay buffer.
        Parameters
        ----------
        size: int
            Max number of transitions to store in the buffer. When the buffer
            overflows the old memories are dropped.
        """
        self._storage = []
        self._maxsize = size
        self._next_idx = 0

    def __len__(self):
        return len(self._storage)

    def add(self, obs_t, action, reward, obs_tp1, done):
        data = (obs_t, action, reward, obs_tp1, done)

        if self._next_idx >= len(self._storage):
            self._storage.append(data)
        else:
            self._storage[self._next_idx] = data
        self._next_idx = (self._next_idx + 1) % self._maxsize

    def _encode_sample(self, idxes):
        obses_t, actions, rewards, obses_tp1, dones = [], [], [], [], []
        for i in idxes:
            data = self._storage[i]
            obs_t, action, reward, obs_tp1, done = data
            obses_t.append(np.array(obs_t, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(reward)
            obses_tp1.append(np.array(obs_tp1, copy=False))
            dones.append(done)
        return np.array(obses_t), np.array(actions), np.array(rewards), np.array(obses_tp1), np.array(dones)

    def sample(self, batch_size):
        """Sample a batch of experiences.
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        """
        idxes = [random.randint(0, len(self._storage) - 1) for _ in range(batch_size)]
        return self._encode_sample(idxes)


class PrioritizedReplayBuffer(ReplayBuffer):
    def __init__(self, size, alpha):
        """Create Prioritized Replay buffer.
        Parameters
        ----------
        size: int
            Max number of transitions to store in the buffer. When the buffer
            overflows the old memories are dropped.
        alpha: float
            how much prioritization is used
            (0 - no prioritization, 1 - full prioritization)
        See Also
        --------
        ReplayBuffer.__init__
        """
        super(PrioritizedReplayBuffer, self).__init__(size)
        assert alpha >= 0
        self._alpha = alpha

        it_capacity = 1
        while it_capacity < size:
            it_capacity *= 2

        self._it_sum = utils.SumSegmentTree(it_capacity)
        self._it_min = utils.MinSegmentTree(it_capacity)
        self._max_priority = 1.0

    def add(self, *args, **kwargs):
        """See ReplayBuffer.store_effect"""
        idx = self._next_idx
        super().add(*args, **kwargs)
        self._it_sum[idx] = self._max_priority ** self._alpha
        self._it_min[idx] = self._max_priority ** self._alpha

    def _sample_proportional(self, batch_size):
        res = []
        p_total = self._it_sum.sum(0, len(self._storage) - 1)
        every_range_len = p_total / batch_size
        for i in range(batch_size):
            mass = random.random() * every_range_len + i * every_range_len
            idx = self._it_sum.find_prefixsum_idx(mass)
            res.append(idx)
        return res

    def sample(self, batch_size, beta):
        """Sample a batch of experiences.
        compared to ReplayBuffer.sample
        it also returns importance weights and idxes
        of sampled experiences.
        Parameters
        ----------
        batch_size: int
            How many transitions to sample.
        beta: float
            To what degree to use importance weights
            (0 - no corrections, 1 - full correction)
        Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        weights: np.array
            Array of shape (batch_size,) and dtype np.float32
            denoting importance weight of each sampled transition
        idxes: np.array
            Array of shape (batch_size,) and dtype np.int32
            idexes in buffer of sampled experiences
        """
        assert beta > 0

        idxes = self._sample_proportional(batch_size)

        weights = []
        p_min = self._it_min.min() / self._it_sum.sum()
        max_weight = (p_min * len(self._storage)) ** (-beta)

        for idx in idxes:
            p_sample = self._it_sum[idx] / self._it_sum.sum()
            weight = (p_sample * len(self._storage)) ** (-beta)
            weights.append(weight / max_weight)
        weights = np.array(weights)
        encoded_sample = self._encode_sample(idxes)
        return tuple(list(encoded_sample) + [weights, idxes])

    def update_priorities(self, idxes, priorities):
        """Update priorities of sampled transitions.
        sets priority of transition at index idxes[i] in buffer
        to priorities[i].
        Parameters
        ----------
        idxes: [int]
            List of idxes of sampled transitions
        priorities: [float]
            List of updated priorities corresponding to
            transitions at the sampled idxes denoted by
            variable `idxes`.
        """
        assert len(idxes) == len(priorities)
        for idx, priority in zip(idxes, priorities):
            assert priority > 0
            assert 0 <= idx < len(self._storage)
            self._it_sum[idx] = priority ** self._alpha
            self._it_min[idx] = priority ** self._alpha

            self._max_priority = max(self._max_priority, priority)


In [ ]:
import operator



class LinearSchedule(object):
    def __init__(self, schedule_timesteps, final_p, initial_p=1.0):
        """Linear interpolation between initial_p and final_p over
        schedule_timesteps. After this many timesteps pass final_p is
        returned.
        Parameters
        ----------
        schedule_timesteps: int
            Number of timesteps for which to linearly anneal initial_p
            to final_p
        initial_p: float
            initial output value
        final_p: float
            final output value
        """
        self.schedule_timesteps = schedule_timesteps
        self.final_p = final_p
        self.initial_p = initial_p

    def value(self, t):
        """See Schedule.value"""
        fraction = min(float(t) / self.schedule_timesteps, 1.0)
        return self.initial_p + fraction * (self.final_p - self.initial_p)

class SegmentTree(object):
    def __init__(self, capacity, operation, neutral_element):
        """Build a Segment Tree data structure.
        https://en.wikipedia.org/wiki/Segment_tree
        Can be used as regular array, but with two
        important differences:
            a) setting item's value is slightly slower.
               It is O(lg capacity) instead of O(1).
            b) user has access to an efficient ( O(log segment size) )
               `reduce` operation which reduces `operation` over
               a contiguous subsequence of items in the array.
        Paramters
        ---------
        capacity: int
            Total size of the array - must be a power of two.
        operation: lambda obj, obj -> obj
            and operation for combining elements (eg. sum, max)
            must form a mathematical group together with the set of
            possible values for array elements (i.e. be associative)
        neutral_element: obj
            neutral element for the operation above. eg. float('-inf')
            for max and 0 for sum.
        """
        assert capacity > 0 and capacity & (capacity - 1) == 0, "capacity must be positive and a power of 2."
        self._capacity = capacity
        self._value = [neutral_element for _ in range(2 * capacity)]
        self._operation = operation

    def _reduce_helper(self, start, end, node, node_start, node_end):
        if start == node_start and end == node_end:
            return self._value[node]
        mid = (node_start + node_end) // 2
        if end <= mid:
            return self._reduce_helper(start, end, 2 * node, node_start, mid)
        else:
            if mid + 1 <= start:
                return self._reduce_helper(start, end, 2 * node + 1, mid + 1, node_end)
            else:
                return self._operation(
                    self._reduce_helper(start, mid, 2 * node, node_start, mid),
                    self._reduce_helper(mid + 1, end, 2 * node + 1, mid + 1, node_end)
                )

    def reduce(self, start=0, end=None):
        """Returns result of applying `self.operation`
        to a contiguous subsequence of the array.
            self.operation(arr[start], operation(arr[start+1], operation(... arr[end])))
        Parameters
        ----------
        start: int
            beginning of the subsequence
        end: int
            end of the subsequences
        Returns
        -------
        reduced: obj
            result of reducing self.operation over the specified range of array elements.
        """
        if end is None:
            end = self._capacity
        if end < 0:
            end += self._capacity
        end -= 1
        return self._reduce_helper(start, end, 1, 0, self._capacity - 1)

    def __setitem__(self, idx, val):
        # index of the leaf
        idx += self._capacity
        self._value[idx] = val
        idx //= 2
        while idx >= 1:
            self._value[idx] = self._operation(
                self._value[2 * idx],
                self._value[2 * idx + 1]
            )
            idx //= 2

    def __getitem__(self, idx):
        assert 0 <= idx < self._capacity
        return self._value[self._capacity + idx]


class SumSegmentTree(SegmentTree):
    def __init__(self, capacity):
        super(SumSegmentTree, self).__init__(
            capacity=capacity,
            operation=operator.add,
            neutral_element=0.0
        )

    def sum(self, start=0, end=None):
        """Returns arr[start] + ... + arr[end]"""
        return super(SumSegmentTree, self).reduce(start, end)

    def find_prefixsum_idx(self, prefixsum):
        """Find the highest index `i` in the array such that
            sum(arr[0] + arr[1] + ... + arr[i - i]) <= prefixsum
        if array values are probabilities, this function
        allows to sample indexes according to the discrete
        probability efficiently.
        Parameters
        ----------
        perfixsum: float
            upperbound on the sum of array prefix
        Returns
        -------
        idx: int
            highest index satisfying the prefixsum constraint
        """
        assert 0 <= prefixsum <= self.sum() + 1e-5
        idx = 1
        while idx < self._capacity:  # while non-leaf
            if self._value[2 * idx] > prefixsum:
                idx = 2 * idx
            else:
                prefixsum -= self._value[2 * idx]
                idx = 2 * idx + 1
        return idx - self._capacity


class MinSegmentTree(SegmentTree):
    def __init__(self, capacity):
        super(MinSegmentTree, self).__init__(
            capacity=capacity,
            operation=min,
            neutral_element=float('inf')
        )

    def min(self, start=0, end=None):
        """Returns min(arr[start], ...,  arr[end])"""

        return super(MinSegmentTree, self).reduce(start, end)


In [ ]:

#Taken from https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py
class OrnsteinUlhenbeckActionNoise(object):
    def __init__(self, mu, sigma=0.2, theta=0.2, dt=5e-2, x0=None):
        self.mu = mu
        self.sigma = sigma
        self.theta = theta
        self.dt = dt
        self.x0 = x0
        self.reset() # reset the noise
        
    def __call__(self):
        x = self.x_prev + self.theta*(self.mu-self.x_prev)*self.dt + \
            self.sigma*np.sqrt(self.dt)*np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x
    
    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)
    
    def __repr__(self):
        return 'OrnsteinUlhenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)





def test_policy(policy):
    episode_rewards = np.zeros(10)
    for i in range(10):
        #reset environment
        #get initial state
        s = env.reset()
        done = False
        #Loop until end of episode
        while not done:
            #get action from policy
            #In testing we do not use exploration noise
            a = policy.get_action(np.array(s))
            a = np.clip(a, env.action_space.low[0], env.action_space.high[0])
            #take step in environment with action
            #observe new state, reward and whether the episode is done
            s_new, r, done, _ = env.step(a)
            #add reward to episode reward
            episode_rewards[i] += r
            #update states
            s = s_new
    #calculate average episode reward over 10 tests
    print(np.mean(episode_rewards))
    return np.mean(episode_rewards)

def main(args):
    #Boolean to specify if we use prioritized experience replay
    #False = original DDPG
    #True = DDPG + PER
    prioritized = True
    POWER = args.power
    channel_no = 1
    #specificy PyBullet environment
    # env_name = "InvertedPendulumBulletEnv-v0"
    # env_name = "Pendulum-v0"
    #env_name = "HopperBulletEnv-v0"
    env_name = "wirelessCF"
    #Set random seed number
    seed = args.seed

    #Output format
    #True = training and testing results
    #False = only testing results are printed
    verbose = True


    #Fixed Parameters
    test_freq = 2000
    train_steps = 100*args.n_episodes
    NUM_EPISODES = args.n_episodes
    Tag = 'WSEEonly'
    #DDPG parameters
    batch_size = 64
    gamma = 0.99
    tau = 0.001
    buffer_size = int(args.buffer_size)

    #PER parameters
    prioritized_replay_alpha=0.6
    prioritized_replay_beta0=0.4
    prioritized_replay_beta_iters=None
    prioritized_replay_eps=1e-6
    if prioritized:
        file_name = "Priority_DDPG_"+env_name[:-12]+"_"+str(seed)
    else:
        file_name = "DDPG_"+env_name[:-12]+"_"+str(seed)

    print('Running: '+file_name)

    # env = gym.make(env_name)
    env = wirelessCF(pu=POWER, nrx=2, seed=seed)
    eval_env = wirelessCF(pu=POWER, nrx=2, seed=seed, eval=True)
    # Set seeds
    # env.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    # environment information
    s_dim = env.observation_space.shape[0]
    a_dim = env.action_space.shape[0]
    a_max = float(env.action_space.high[0])

    #Create DDPG policy
    policy = DDPG(s_dim, a_dim, a_max, args)

    #If we are not doing prioritized experience replay
    #Then we use my implementation of the uniform replay buffer
    if not prioritized:
        replay_buffer = ReplayBuffer_org()
        beta_schedule = None
    #If we are doing prioritized experience replay
    #Then we use the OpenAI implementation
    else:
        replay_buffer = PrioritizedReplayBuffer(buffer_size, prioritized_replay_alpha)
        if prioritized_replay_beta_iters is None:
            prioritized_replay_beta_iters = train_steps
        #Create annealing schedule
        beta_schedule = LinearSchedule(prioritized_replay_beta_iters, initial_p=prioritized_replay_beta0, final_p=1.0)

    #test policy (at this point policy is just random)
    # test_scores = [test_policy(policy)]
    #Save data file (used for plotting)
    # np.save("data/%s" % (file_name), test_scores)

    OUnoise = OrnsteinUlhenbeckActionNoise(mu=np.zeros(env.action_space.shape[0]))

    total_time = 0
    test_time = 0
    episode = 0
    done = True

    t_0 = time.time()

    while total_time < train_steps:

        if done:

            if total_time != 0:
                if verbose:
                    print("Total Time Steps: "+str(total_time)+ " Episode Reward: "+str(episode_r)+" Runtime: "+str(int(time.time() - t_0)))

                #set beta value used for importance sampling weights
                beta_value = 0
                if prioritized:
                    beta_value = beta_schedule.value(total_time)

                #train DDPG
                policy.train(replay_buffer, prioritized, beta_value, prioritized_replay_eps, episode_t, args.batch_size, args.gamma, args.tau)

            #Check if we need to need to test DDPG
            # if test_time >= test_freq:
            #     test_time %= test_freq
            #     test_scores.append(test_policy(policy))
            #     np.save("data/%s" % (file_name), test_scores)

            # evaluation
            eval_s = eval_env.reset()
            eval_done = False
            while not eval_done:
                eval_a = policy.get_action(np.array(eval_s))
                eval_a = np.clip(eval_a, eval_env.action_space.low[0], eval_env.action_space.high[0])
                eval_s, eval_r, eval_done, _ = eval_env.step(eval_a)
                

            #reset environment
            #get intial state
            #reset episode statistics
            s = env.reset()
            OUnoise.reset()
            done = False
            episode_r = 0
            episode_t = 0
            episode += 1


        #Given current state, get action
        a = policy.get_action(np.array(s))
        #Apply exploration noise to action
        # a = (a + np.random.normal(0, 0.1, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
        a = a + OUnoise()
        a = np.clip(a, env.action_space.low, env.action_space.high)
        #Using action, take step in environment, observe new state, reward and episode status
        s_new, r, done, _ = env.step(a)
        done_bool = 0 if episode_t + 1 == env._max_episode_steps else float(done)
        episode_r += r

        # Store data in replay buffer
        if not prioritized:
            replay_buffer.add((s, a, r, s_new, done_bool))
        else:
            replay_buffer.add(s, a, r, s_new, done_bool)

        #update state and episode statistics
        s = s_new
        episode_t += 1
        total_time += 1
        test_time += 1

    #Final policy test
    # test_scores.append(test_policy(policy))
    # #Save data file (used for plotting)
    # np.save("data/%s" % (file_name), test_scores)
    # print(test_scores)
    config = 'lr_a'+str(args.lr_a)+'lr_c'+str(args.lr_c)+'tau_'+str(args.tau)+'gamma_'+str(args.gamma)+'_batch_'+str(args.batch_size)+'_layers'+str(args.fc1_dims)+'_'+str(args.fc2_dims)+'_'+str(args.fc3_dims)
    path = Path('testlogs_'+'power_'+str(POWER)+'_channel_'+str(channel_no)+'_'+str(Tag)+'_config_'+str(config))
    dest = Path(str(path))
    dest.mkdir(parents=True, exist_ok=True)
    # dest = 'testlogs'
    import pandas as pd
    ########################## Random Scheme ##################################
    temp = env.pi
    random_WSEE = []
    # random_SEs = []
    random_sum_SE = []
    # random_constraint_value = []
    for i in range(NUM_EPISODES):
        action = np.random.uniform(0,1,(env.action_space.shape[0]))
        env.pi.append(action)
        next_state = env.cal_state()
        reward = env.cal_reward(0)
        # next_state = np.reshape(next_state, [1, self.observation_space.shape[0]])
        done = False
        # random_SEs.append(env.SEs.tolist())
        random_sum_SE.append(env.cal_sum_SE(0))
        random_WSEE.append(env.cal_total_WSEE(0))
        # print(random_sum_SE, random_WSEE)
        # print()
    
    env.pi = temp
    powerdbm = POWER

    rand_df_wsee = pd.DataFrame(data=random_WSEE, columns=['WSEE'])
    rand_df_wsee.to_csv(str(dest)+'/'+str(Tag)+'rand_WSEE'+str(powerdbm)+'.csv')

    # rand_data_se = pd.DataFrame(data=random_SEs,columns=['SE'+str(i) for i in range(1,env.K+1)])
    # rand_data_se.to_csv(str(dest)+'/'+str(Tag)+'rand_SEs'+str(powerdbm)+'.csv')

    rand_df_sumSE = pd.DataFrame(data=random_sum_SE, columns=['sum_SE'])
    rand_df_sumSE.to_csv(str(dest)+'/'+str(Tag)+'rand_sum_SE'+str(powerdbm)+'.csv')

    ###########################################################################
    episode = [i for i in range(NUM_EPISODES)]
    eval_episode = [i for i in range(len(eval_env.WSEE_history))]
    powerdbm = POWER

    df_wsee = pd.DataFrame(data=env.WSEE_history, columns=['WSEE'])
    df_wsee.to_csv(str(dest)+'/'+str(Tag)+'WSEE'+str(powerdbm)+'.csv')

    df_eval_wsee = pd.DataFrame(data=eval_env.WSEE_history, columns=['WSEE'])
    df_eval_wsee.to_csv(str(dest)+'/'+str(Tag)+'eval_WSEE'+str(powerdbm)+'.csv')

    df_fpawsee = pd.DataFrame(data=env.FPA_WSEE_history, columns=['WSEE'])
    df_fpawsee.to_csv(str(dest)+'/'+str(Tag)+'FPA_WSEE'+str(powerdbm)+'.csv')

    df_reward = pd.DataFrame(data=env.score_history, columns=['reward'])
    df_reward.to_csv(str(dest)+'/'+str(Tag)+'reward'+str(powerdbm)+'.csv')

    df_eval_reward = pd.DataFrame(data=eval_env.score_history, columns=['reward'])
    df_eval_reward.to_csv(str(dest)+'/'+str(Tag)+'eval_reward'+str(powerdbm)+'.csv')

    df_sumSE = pd.DataFrame(data=env.sumSE_history, columns=['sum_SE'])
    df_sumSE.to_csv(str(dest)+'/'+str(Tag)+'sum_SE'+str(powerdbm)+'.csv')

    df_eval_sumSE = pd.DataFrame(data=eval_env.sumSE_history, columns=['sum_SE'])
    df_eval_sumSE.to_csv(str(dest)+'/'+str(Tag)+'eval_sum_SE'+str(powerdbm)+'.csv')

    df_fpasumSE = pd.DataFrame(data=env.FPA_sumSE_history, columns=['sum_SE'])
    df_fpasumSE.to_csv(str(dest)+'/'+str(Tag)+'FPA_sum_SE'+str(powerdbm)+'.csv')

    df_time_per_episode = pd.DataFrame(data=env.time_per_episode, columns=['time'])
    df_time_per_episode.to_csv(str(dest)+'/'+str(Tag)+'time_per_episode'+str(powerdbm)+'.csv')
    
    def numpy_ewma_vectorized_v2(data, window):

        alpha = 2 /(window + 1.0)
        alpha_rev = 1-alpha
        n = data.shape[0]

        pows = alpha_rev**(np.arange(n+1))

        scale_arr = 1/pows[:-1]
        offset = data[0]*pows[1:]
        pw0 = alpha*alpha_rev**(n-1)

        mult = data*pw0*scale_arr
        cumsums = mult.cumsum()
        out = offset + cumsums*scale_arr[::-1]
        return out

    smooth_window = 20

    plt.figure(figsize=(15,10))
    reward = numpy_ewma_vectorized_v2(np.array(env.score_history),smooth_window)
    episode = np.array(episode)

    plt.plot(episode, reward,'-' ,label='Reward')
    plt.title('Training reward over multiple runs')
    plt.xlabel('Number of episodes')
    plt.ylabel('Cumulative reward')
    plt.legend()
    plt.savefig(dest/r'reward.jpg', dpi=300)
    # plt.show()



    plt.figure(figsize=(15,10))
    out_WSEE = numpy_ewma_vectorized_v2(np.array(env.WSEE_history),smooth_window)
    FPA_WSEE = numpy_ewma_vectorized_v2(np.array(env.FPA_WSEE_history),smooth_window)
    rand_WSEE = numpy_ewma_vectorized_v2(np.array(random_WSEE),smooth_window)

    plt.plot(episode, out_WSEE,'-' ,label='DDPG '+str(POWER)+'dBm')
    plt.plot(episode, FPA_WSEE,'-' ,label='FPA '+str(POWER)+'dBm')
    plt.plot(episode, rand_WSEE,'-' ,label='Random Scheme '+str(POWER)+'dBm')
    plt.title('WSEE: DDPG vs FPA vs Random Scheme')
    plt.xlabel('Number of episodes')
    plt.ylabel('WSEE')
    plt.legend()
    plt.savefig(dest/r'WSEE.jpg', dpi=300)
    # plt.show()

    plt.figure(figsize=(15,10))
    out_eval_WSEE = numpy_ewma_vectorized_v2(np.array(eval_env.WSEE_history),smooth_window)
    FPA_eval_WSEE = numpy_ewma_vectorized_v2(np.array(eval_env.FPA_WSEE_history),smooth_window)
    rand_WSEE = numpy_ewma_vectorized_v2(np.array(random_WSEE),smooth_window)

    plt.plot(eval_episode, out_eval_WSEE,'-' ,label='DDPG '+str(POWER)+'dBm')
    plt.plot(eval_episode, FPA_eval_WSEE,'-' ,label='FPA '+str(POWER)+'dBm')
    plt.plot(episode, rand_WSEE,'-' ,label='Random Scheme '+str(POWER)+'dBm')
    plt.title(' WSEE Evalutation: DDPG vs FPA vs Random Scheme')
    plt.xlabel('Number of episodes')
    plt.ylabel('WSEE')
    plt.legend()
    plt.savefig(dest/r'eval_WSEE.jpg', dpi=300)
    # plt.show()


    plt.figure(figsize=(15,10))
    out_sum_SE = numpy_ewma_vectorized_v2(np.array(env.sumSE_history),smooth_window)
    FPA_sum_SE = numpy_ewma_vectorized_v2(np.array(env.FPA_sumSE_history),smooth_window)
    rand_sum_SE = numpy_ewma_vectorized_v2(np.array(random_sum_SE),smooth_window)

    plt.plot(episode, out_sum_SE,'-' ,label='DDPG '+str(POWER)+'dBm')
    plt.plot(episode, FPA_sum_SE,'-' ,label='FPA '+str(POWER)+'dBm')
    plt.plot(episode, rand_sum_SE,'-' ,label='Random Scheme '+str(POWER)+'dBm')
    plt.title('sum SE: DDPG vs FPA vs Random Scheme')
    plt.xlabel('Number of episodes')
    plt.ylabel('sum SE')
    plt.legend()
    plt.savefig(dest/r'sum_SE.jpg', dpi=300)
    # plt.show()

    plt.figure(figsize=(15,10))
    out_eval_sum_SE = numpy_ewma_vectorized_v2(np.array(eval_env.sumSE_history),smooth_window)
    FPA_eval_sum_SE = numpy_ewma_vectorized_v2(np.array(eval_env.FPA_sumSE_history),smooth_window)
    rand_sum_SE = numpy_ewma_vectorized_v2(np.array(random_sum_SE),smooth_window)

    plt.plot(eval_episode, out_eval_sum_SE,'-' ,label='DDPG '+str(POWER)+'dBm')
    plt.plot(eval_episode, FPA_eval_sum_SE,'-' ,label='FPA '+str(POWER)+'dBm')
    plt.plot(episode, rand_sum_SE,'-' ,label='Random Scheme '+str(POWER)+'dBm')
    plt.title('sum SE Evaluation: DDPG vs FPA vs Random Scheme')
    plt.xlabel('Number of episodes')
    plt.ylabel('sum SE')
    plt.legend()
    plt.savefig(dest/r'eval_sum_SE.jpg', dpi=300)
    # plt.show()
    print('Mean time per episode: ', df_time_per_episode.stack().mean())
    print('Std of time: ', df_time_per_episode.stack().std())
    print('Max: ', df_time_per_episode.stack().max())
    print('Min: ', df_time_per_episode.stack().min())




In [ ]:
# main()
import argparse
# if __name__ == '__main__':
parser = argparse.ArgumentParser(
    description='Commandline utility for training RL Models'
)
parser.add_argument('-seed', type=int, default=143, help='Seed value')
parser.add_argument('-n_episodes', type=int, default=2500, help='Number of episodes to run for')
parser.add_argument('-lr_a', type=float, default=1e-7, help='Learning rate for actor')
parser.add_argument('-lr_c', type=float, default=1e-5, help='Learning rate for critic')
parser.add_argument('-tau', type=float, default=1e-5, help='Polyak averaging factor')
parser.add_argument('-batch_size', type=int, default=40, help='Batch size for replay memory sampling')
parser.add_argument('-gamma', type=float, default=0.6, help='Discount factor')
parser.add_argument('-buffer_size', type=int, default=1e6, help='Maximum replay memory size')
parser.add_argument('-fc1_dims', type=int, default=500, help='Hidden Layer 1 dimensions')
parser.add_argument('-fc2_dims', type=int, default=400, help='Hidden Layer 2 dimensions')
parser.add_argument('-fc3_dims', type=int, default=300, help='Hidden Layer 3 dimensions')
parser.add_argument('-power', type=int, default=30, help='Power in dBm')
# args = parser.parse_args()
args = parser.parse_known_args()[0]

# args.dims = [args.dims]
total_time_start = time.time()
main(args)
total_time_end = time.time()
print('Total time taken: ', total_time_end - total_time_start)
